In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark =  SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 18:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/05 18:24:37 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [35]:
data = [(1,"Smith",-1,"2018","10","M",3000),
         (2,"Rose",1,"2010","20","M",4000),
        (3,"Williams",1,"2010","10","M",1000),
        (4,"Jones",2,"2005","10","F",2000),
        (5,"Brown",2,"2010","40","",-1),
        (6,"Smith Brown",-1,"2010","50","",-1)]

In [36]:
empColumns = ["emp_id","name","superior_emp_id","year_joiner","emp_dept_id","gender","salary"]

In [37]:
empdf = spark.createDataFrame(data=data,schema=empColumns)

In [38]:
empdf.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joiner: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [39]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]

In [40]:
deptdf = spark.createDataFrame(data=dept,schema = ["dept_name","dept_id"])

In [41]:
deptdf.show()
empdf.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+

+------+-----------+---------------+-----------+-----------+------+------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|
+------+-----------+---------------+-----------+-----------+------+------+
|     1|      Smith|             -1|       2018|         10|     M|  3000|
|     2|       Rose|              1|       2010|         20|     M|  4000|
|     3|   Williams|              1|       2010|         10|     M|  1000|
|     4|      Jones|              2|       2005|         10|     F|  2000|
|     5|      Brown|              2|       2010|         40|      |    -1|
|     6|Smith Brown|             -1|       2010|         50|      |    -1|
+------+-----------+---------------+-----------+-----------+------+------+



In [42]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [43]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"left").show()

+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|     1|      Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|       Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|   Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|      Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|      Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|Smith Brown|             -1|       2010|         50|      |    -1|     NULL|   NULL|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+



In [44]:
empdf.createOrReplaceTempView("employee")
deptdf.createOrReplaceTempView("dept")

In [45]:
spark.sql(""" select emp_id, name from employee e join dept d on e.emp_dept_id == d.dept_id """).show()

+------+--------+
|emp_id|    name|
+------+--------+
|     1|   Smith|
|     3|Williams|
|     4|   Jones|
|     2|    Rose|
|     5|   Brown|
+------+--------+



In [52]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"left").show()

+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|     1|      Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|       Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|   Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|      Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|      Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|Smith Brown|             -1|       2010|         50|      |    -1|     NULL|   NULL|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+



In [55]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"right").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  NULL|    NULL|           NULL|       NULL|       NULL|  NULL|  NULL|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [56]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"outer").show()

+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|     1|      Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|   Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|      Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|       Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  NULL|       NULL|           NULL|       NULL|       NULL|  NULL|  NULL|    Sales|     30|
|     5|      Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|Smith Brown|             -1|       2010|         50|      |    -1|     NULL|   NULL|
+------+-----------+---------------+-----------+-----------+------+---

In [58]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"leftouter").show()

+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+
|     1|      Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|       Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|   Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|      Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|      Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|Smith Brown|             -1|       2010|         50|      |    -1|     NULL|   NULL|
+------+-----------+---------------+-----------+-----------+------+------+---------+-------+



In [59]:
empdf.join(deptdf,empdf.emp_dept_id == deptdf.dept_id,"leftsemi").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [60]:
empdf.join(deptdf,empdf.emp_dept_id ==  deptdf.dept_id,"leftanti").show()

+------+-----------+---------------+-----------+-----------+------+------+
|emp_id|       name|superior_emp_id|year_joiner|emp_dept_id|gender|salary|
+------+-----------+---------------+-----------+-----------+------+------+
|     6|Smith Brown|             -1|       2010|         50|      |    -1|
+------+-----------+---------------+-----------+-----------+------+------+



In [61]:
empdf.createOrReplaceTempView("employee")
deptdf.createOrReplaceTempView("dept")

In [64]:
spark.sql(""" select name,year_joiner from employee left join dept on dept.dept_id = employee.emp_dept_id """).show()

+-----------+-----------+
|       name|year_joiner|
+-----------+-----------+
|      Smith|       2018|
|       Rose|       2010|
|   Williams|       2010|
|      Jones|       2005|
|      Brown|       2010|
|Smith Brown|       2010|
+-----------+-----------+

